In [1]:
# import dependencies
# import pandas as pd
# import numpy as np
import random
import json


In [2]:
# select desired state
state = 'missouri'

# access json file with data to be sonified for avaliable states
stateData = json.load(open('../resources/stateData.json'))

# list to hold probabilities for which demographic a new case will belong to
infectProbs = []
# list to hold probability that an infection will result in death for each demographic
deathProbs = []

streamProbs = []

# populate lists above
for data in stateData[state]:
    infectProbs.append(stateData[state][data]['chance_of_infection'])
    deathProbs.append(stateData[state][data]['chance_of_death'])
    streamProbs.append(stateData[state][data]['inverted_ccvi'])


In [3]:
# function to determine how many cases correspond to a particular threshold
def nthThresh(n,thresh):
    return 100 * n + thresh

# function to adjust spreadProbs weights and calculate r0 for corresponding threshold
def newThresh():
    global n
    global thresh
    if n == 1:
        spreadProbs[3] -= 1
        spreadProbs[2] += 1
    elif n == 2:
        spreadProbs[3] -= 1
        spreadProbs[1] += 1
    elif n == 13:
        spreadProbs[2] -= 1
        spreadProbs[0] += 1
    elif n == 14:
        spreadProbs[1] -= 1
        spreadProbs[0] += 1
    elif n != 0 and n < 13:
        spreadProbs[3] -= 1
        spreadProbs[0] += 1
    r0 = (spreadProbs[1] + 2*spreadProbs[2] + 3*spreadProbs[3]) / sum(spreadProbs)
    # calculate new threshold
    thresh = nthThresh(n, thresh)
    # add one to threshold n value
    n += 1
    
def newTrans():
    global trans
    t = random.randint(1,12)
    if trans+t > 12:
        trans -= t
    elif trans-t < -12:
        trans += t
    else:
        trans += t*random.choice([-1,1])

# function to select rescaled incubation period as a delay in milliseconds.
def incubate(n):
    if n == 1:  
        sub = 3
        return random.randint(0, sub*40)*2000/sub
    elif n == 2:
        sub = random.choice([3,4])
        return random.randint(0, sub*40)*2000/sub
    elif n == 3:
        sub = random.choice([6,4])
        return random.randint(0, sub*40)*2000/sub
    elif n == 4:
        sub = random.choice([6,8])
        return random.randint(0, sub*40)*2000/sub
    elif n == 5:
        sub = random.choice([6,8,5])
        return random.randint(0, sub*40)*2000/sub
    else:
        sub = random.choice([6,8,5,7])
        return random.randint(0, sub*40)*2000/sub

# function to select a value based on a probability distribution array
def select(probsArray):
    return random.choices(list(range(len(probsArray))),weights=probsArray)[0]

# function to decide between one and zero based on probability that 1 will be chosen
def decide(prob):
    return random.choices([0,1], weights=[100-prob, prob])[0]

def perspective(time):
    probs = [0, 0, 0, 0, 0, 0]
    for demo in range(0, len(time), 3):
        probs[time[demo]] += streamProbs[time[demo]]
    return select(probs)

# function to generate midi notes
def midi(trans, death):
    note = random.choice(midiNotes)
    return note + trans - death



In [4]:
infectProbs

In [5]:
streamProbs

In [6]:
# weighted probabilities for #people the current case can spread infection to
# (from left to right, values refer to weights for 0, 1, 2, and 3 new infections)
spreadProbs = [0,0,0,12]
# symptomatic cases that have occured
sympCount = 0
# deaths that have occured
deathCount = 0
# case count threshold for when new r0 shoud be calculated
thresh = 0
# nth threshold
n = 1
# infection rate
r0 = 3
# transposition in semitones
trans = 0
# cases remaining (one case to start it off)
times = []
# which case
case = 0
# array of midi notes
midiNotes = [48, 51, 55, 58, 60, 62, 63, 65, 67, 68, 70, 74, 77, 80]
# dictionary with all data
weathering = {
    'time': [],
    'demo': [],
    'note': [],
    'death': [],
    'n': [],
}


In [7]:
times.append(incubate(n))

while len(times) > 0:
    
    # determine how much time has elapse since beginning of piece
    now = min(times)
    
    # if symptomatic add case info to weathering
    if decide(40) == 0:

        # add current case to total # of cases
        # (only sympotmatic cases adjust threshold)
        sympCount += 1
        
        if sympCount % 50 == 0:
            newTrans()

         # if case count is at or over threshold number
        if sympCount > nthThresh(n, thresh):
            newThresh()
            
        weathering['time'].append(now)

        # select which demographic the current case belongs to
        demo = select(infectProbs)
        weathering['demo'].append(demo)

        # decide/record if case results in death or not
        death = decide(deathProbs[demo])
        weathering['death'].append(death)

        # select which note
        weathering['note'].append(midi(trans, death))

        weathering['n'].append(n)
    
   

    # decide how many people the current case infects and add them to more
    for i in range(select(spreadProbs)):
        
        # select incubation period (delay time)
        delay = incubate(n)
        
        # determine time elapsed since beginning and append to times list
        time = now + delay
        times.append(time)

    # remove current case from timestamps of remaining cases
    times.pop(times.index(now))


In [8]:
result = sorted(list(zip(weathering['time'], weathering['demo'], weathering['note'], weathering['death'], weathering['n'])))

len(result)


11665

In [9]:
# result

[(37333.333333333336, 1, 48, 0, 1),
 (89333.33333333333, 5, 65, 0, 1),
 (104666.66666666667, 5, 63, 0, 1),
 (108000.0, 0, 60, 0, 1),
 (117999.99999999999, 2, 63, 0, 1),
 (127333.33333333333, 5, 80, 0, 1),
 (127999.99999999999, 5, 74, 0, 1),
 (129999.99999999999, 2, 74, 0, 1),
 (130666.66666666666, 0, 70, 0, 1),
 (131333.33333333334, 5, 74, 0, 1),
 (132000.0, 0, 48, 0, 1),
 (138666.66666666666, 1, 60, 0, 1),
 (140000.0, 1, 58, 0, 1),
 (146666.66666666666, 0, 51, 0, 1),
 (148000.0, 5, 55, 0, 1),
 (149333.3333333333, 0, 60, 0, 1),
 (149333.33333333334, 0, 51, 0, 1),
 (150666.66666666666, 5, 51, 0, 1),
 (152000.0, 2, 62, 0, 1),
 (152000.0, 5, 70, 0, 1),
 (154666.66666666666, 0, 58, 0, 1),
 (156000.0, 5, 51, 0, 1),
 (156666.6666666667, 3, 48, 0, 1),
 (158000.0, 3, 63, 0, 1),
 (158666.66666666666, 5, 55, 0, 1),
 (159333.33333333334, 5, 63, 0, 1),
 (160000.0, 1, 63, 0, 1),
 (160000.0, 3, 63, 0, 1),
 (162000.0, 4, 55, 0, 1),
 (163333.33333333334, 5, 51, 0, 1),
 (164666.66666666666, 3, 70, 0, 1

In [10]:
# for i, r in enumerate(result):
#     print(f'{i}: {r[4]}')



In [11]:
meep = float(round(result[3][0],6))
moop = float(round(result[4][0],6))
print(meep)
print(moop)

moop - meep


108000.0
118000.0


10000.0

In [118]:

meow = [1,2,3,4,5,6,7,8]

gug = [1, 2]

meow[3:len(meow):4] = [','.join(list(map(str, gug)))]*len(meow[3:len(meow):4])

meow








[1, 2, 3, '1,2', 5, 6, 7, '1,2']

In [125]:
timing = {}
ms = 0
infection = -1

for case in result:
    
    time = round(case[0], 6)

    if time != ms:
        
        if infection != -1:
            
            demoNotes = timing[infection]['demoNotes']
            demoNotes[3:len(demoNotes):4] = [select(demoWeights)]*len(demoNotes[3:len(demoNotes):4])
        

        
        demoWeights = [0,0,0,0,0,0]
        demoWeights[case[1]] += streamProbs[case[1]]
        
        
        infection += 1
        timing[infection] = {'demoNotes': [case[1], case[2], case[3], 'stream']}
        timing[infection]['delay'] = round(time - ms, 6)
        ms = time
        

    else:
        demoWeights[case[1]] += streamProbs[case[1]]
        timing[infection]['demoNotes'].extend(list(case[1:4])+["stream"])


In [126]:
timing



{0: {'demoNotes': [1, 48, 0, 1], 'delay': 37333.333333},
 1: {'demoNotes': [5, 65, 0, 5], 'delay': 52000.0},
 2: {'demoNotes': [5, 63, 0, 5], 'delay': 15333.333334},
 3: {'demoNotes': [0, 60, 0, 0], 'delay': 3333.333333},
 4: {'demoNotes': [2, 63, 0, 2], 'delay': 10000.0},
 5: {'demoNotes': [5, 80, 0, 5], 'delay': 9333.333333},
 6: {'demoNotes': [5, 74, 0, 5], 'delay': 666.666667},
 7: {'demoNotes': [2, 74, 0, 2], 'delay': 2000.0},
 8: {'demoNotes': [0, 70, 0, 0], 'delay': 666.666667},
 9: {'demoNotes': [5, 74, 0, 5], 'delay': 666.666666},
 10: {'demoNotes': [0, 48, 0, 0], 'delay': 666.666667},
 11: {'demoNotes': [1, 60, 0, 1], 'delay': 6666.666667},
 12: {'demoNotes': [1, 58, 0, 1], 'delay': 1333.333333},
 13: {'demoNotes': [0, 51, 0, 0], 'delay': 6666.666667},
 14: {'demoNotes': [5, 55, 0, 5], 'delay': 1333.333333},
 15: {'demoNotes': [0, 60, 0, 0, 0, 51, 0, 0], 'delay': 1333.333333},
 16: {'demoNotes': [5, 51, 0, 5], 'delay': 1333.333334},
 17: {'demoNotes': [2, 62, 0, 2, 5, 70, 0, 

In [ ]:
with open("../resources/weathering.json", "w") as outfile:
    json.dump(timing, outfile)
    